In [2]:
#Imports
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
import pandas as pd                 
import matplotlib.pyplot as plt    
import numpy as np 

In [3]:
#df_dados = pd.read_csv('credito.csv', sep = ';')
df_dados = pd.read_csv('Dados2018a2019.csv', sep = ';')

In [4]:
#Análise exploratório para verificar colunas TARGET, REMOVER, TRANSFORMA TEXTO EM NÚMERO
#ATENÇÃO POIS O NOME DO CAMPO APARECE APÓS A RELAÇÃO DE VALORES

#TARGET:
#AtrasoPrimeiroPagamento

#REMOVER:
#'Cliente_Id','Cpf','Nome','Bairro','Cidade','Uf','Fazendeiro'

#TRANSFORMAR TEXTO EM NUMERO:

#'EstadoCivil','Sexo','Escolaridade','TipoMoradia','Filho','Classe','Trabalha','TempoTrabalho','TempoTrabalho','Nacionalidade',
#'SpcPrimeiraConsultaScore','SpcPrimeiraConsultaRestricao','Nascimento','PrimeiraCompra','UltimaCompra','IdadePrimeiraCompra',
#'Renda','Risco','Qualidade','Satisfacao','Renegociacao','LimiteCredito','Funcionario','AbertoSituacao','Cargo','SpcHojeRestricao',
#VERIFICA PELO VALORES DISTINCT DE CADA CAMPO COMO PROCEDER
for x in df_dados.columns:
    print(df_dados[x].drop_duplicates())

0        DOURADOS                                      ...
1        Centro                                        ...
2        Progresso                                     ...
4        CENTRO                                        ...
6        Residencial Jardim Europa II                  ...
                               ...                        
45267    SOLAR DE BURITIS                              ...
45283    PRINSECA ISABEL I                             ...
45303    RESIDENCIAL PIETRA                            ...
45308    LOTEAMENTO AROUDO CRUZ                        ...
45322    JUAN DE AYOLAS                                ...
Name: Bairro, Length: 3855, dtype: object
0        Pirajuba                                      ...
1        Ituiutaba                                     ...
4        Tocantinópolis                                ...
8        Patos de Minas                                ...
12       FRUTAL                                        ...
              

0         0
7         7
67        8
94       12
101       6
106       3
136       4
168      10
233       2
329       5
351       1
1265      9
2417     20
3565     15
3662     11
5345     16
5374     14
5860     13
10960    17
13116    21
13120    18
18497    24
Name: AbertoQtdCano, dtype: int64
0           0.00
2         398.54
7        1691.68
8         139.97
26        148.27
          ...   
45329      50.99
45337     134.88
45339      90.46
45341     266.66
45342      89.97
Name: AbertoVlrAtraso, Length: 14752, dtype: float64
0         0
2         7
8         1
26        4
55        5
67        8
74        3
94       12
101       6
120       9
160       2
168      10
395      11
471      13
1446     15
2341     17
2402     16
2417     20
3181     14
4294     24
4975     29
5256     18
5418     21
6130     23
6185     19
6511     38
10088    22
11676    27
22891    41
27179    26
Name: AbertoQtdAtraso, dtype: int64
0          0.00
3        139.97
4        177.84
21       523.05
24

In [5]:
#Pré-processamento 
#Feature selection 
#Removendo campos identificadores
#df_dados = df_dados.drop(['Cliente_Id', 'Cpf', 'Nome', 'Bairro', 'Cidade', 'Uf', 'Fazendeiro'], axis=1)
df_dados = df_dados.drop(['Bairro', 'Cidade', 'Uf', 'Fazendeiro'], axis=1)

In [6]:
df_dados.head()

,EstadoCivil,Sexo,Escolaridade,TipoMoradia,Filho,Classe,Trabalha,TempoTrabalho,SpcHojeRestricao,SpcPrimeiraConsultaScore,...,PrimeiroLimiteCredito_Ind,LimiteCredito_Ind,IdadeAtual_Ind,IdadePrimeiraCompra_Ind,Renda_Ind,PrazoMediaAtraso,PrazoMedioVenda,VlrDh,VlrCartaoCredito,VlrCartaoDebito
0,Solteiro,Feminino,Superior-Completo,Com os pais,Sim,APOSENTADO,Trabalha,10 ANOS,Sem Restrição,C,...,350.0,0.0,36,35,1600.0,50,2,7.68,0.0,0.0
1,Casado,Feminino,Superior-Completo,Alugada,Sim,AUTONOMO/FEIRANTE,Trabalha,10 ANOS,Sem Restrição,A,...,300.0,375.0,31,30,1200.0,4,1,0.00,0.0,0.0
2,Solteiro,Masculino,Médio-Completo,Alugada,Sim,FUNCIONARIO,Trabalha,5 ANOS,Com Restrição,F,...,300.0,0.0,28,27,1258.0,1,2,1.84,0.0,0.0
3,Solteiro,Feminino,Médio-Incompleto,Com os pais,Sim,COMERCIARIO,Trabalha,6 ANOS 5 MESES,Sem Restrição,C,...,350.0,438.0,26,25,1379.0,0,2,0.00,0.0,0.0
4,Casado,Feminino,Médio-Completo,Própria,Não,FUNCIONARIO,Trabalha,12 ANOS 5 MESES,Sem Restrição,C,...,0.0,8000.0,40,39,2000.0,0,5,0.00,0.0,0.0


In [7]:
#colunas categóricas que terão valores por categoria convertidas em número (talvez reavaliar campos data)
#Cria dicionário com colunas categóricas qcolunas categóricas que terão valores por categoria convertidas em número 
#colunas_texto = ['EstadoCivil','Sexo','Escolaridade','TipoMoradia','Filho','Classe','Trabalha','TempoTrabalho','TempoTrabalho','SpcPrimeiraConsultaScore','SpcPrimeiraConsultaRestricao','Nascimento','PrimeiraCompra','UltimaCompra','IdadePrimeiraCompra','Renda','Risco','Qualidade','Satisfacao','Renegociacao','LimiteCredito','Funcionario','AbertoSituacao','Cargo','SpcHojeRestricao','Nacionalidade']
colunas_texto = ['EstadoCivil','Sexo','Escolaridade','TipoMoradia','Filho','Classe','Trabalha','TempoTrabalho','TempoTrabalho','SpcPrimeiraConsultaScore','SpcPrimeiraConsultaRestricao','Nascimento','PrimeiraCompra','UltimaCompra','IdadePrimeiraCompra','Renda','Risco','Qualidade','Satisfacao','Renegociacao','LimiteCredito','Funcionario','AbertoSituacao','Cargo','SpcHojeRestricao','Nacionalidade']
colunas_texto

['EstadoCivil',
 'Sexo',
 'Escolaridade',
 'TipoMoradia',
 'Filho',
 'Classe',
 'Trabalha',
 'TempoTrabalho',
 'TempoTrabalho',
 'SpcPrimeiraConsultaScore',
 'SpcPrimeiraConsultaRestricao',
 'Nascimento',
 'PrimeiraCompra',
 'UltimaCompra',
 'IdadePrimeiraCompra',
 'Renda',
 'Risco',
 'Qualidade',
 'Satisfacao',
 'Renegociacao',
 'LimiteCredito',
 'Funcionario',
 'AbertoSituacao',
 'Cargo',
 'SpcHojeRestricao',
 'Nacionalidade']

In [8]:
#loop que percorre lista de colunas categóricas que terão valores por categoria convertidas em número 
novo_valor_numerico = 0
i = 0
for x in colunas_texto:
    for y in df_dados[x].drop_duplicates():
        df_dados[x].replace({y: str(i)}, inplace=True)
        i+=1
    i = 0

In [11]:
#Trantando dados missing
df_dados.isna()

,EstadoCivil,Sexo,Escolaridade,TipoMoradia,Filho,Classe,Trabalha,TempoTrabalho,SpcHojeRestricao,SpcPrimeiraConsultaScore,...,PrimeiroLimiteCredito_Ind,LimiteCredito_Ind,IdadeAtual_Ind,IdadePrimeiraCompra_Ind,Renda_Ind,PrazoMediaAtraso,PrazoMedioVenda,VlrDh,VlrCartaoCredito,VlrCartaoDebito
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45344,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
45345,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
45346,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
45347,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [12]:
#Se os valores missing mesmos substituídos estiverem influenciando a análise, experimetar df_dados.dropna() ou 
#df_dados.interpolate(method ='linear', limit_direction ='forward') 
df_dados = df_dados.replace(to_replace = np.nan, value = -99) 

In [13]:
df_dados.head()

,EstadoCivil,Sexo,Escolaridade,TipoMoradia,Filho,Classe,Trabalha,TempoTrabalho,SpcHojeRestricao,SpcPrimeiraConsultaScore,...,PrimeiroLimiteCredito_Ind,LimiteCredito_Ind,IdadeAtual_Ind,IdadePrimeiraCompra_Ind,Renda_Ind,PrazoMediaAtraso,PrazoMedioVenda,VlrDh,VlrCartaoCredito,VlrCartaoDebito
0,0,0,0,0,0,0,0,0,0,0,...,350.0,0.0,36,35,1600.0,50,2,7.68,0.0,0.0
1,1,0,0,1,0,1,0,0,0,1,...,300.0,375.0,31,30,1200.0,4,1,0.00,0.0,0.0
2,0,1,1,1,0,2,0,1,1,2,...,300.0,0.0,28,27,1258.0,1,2,1.84,0.0,0.0
3,0,0,2,0,0,3,0,2,0,0,...,350.0,438.0,26,25,1379.0,0,2,0.00,0.0,0.0
4,1,0,1,2,1,2,0,3,0,0,...,0.0,8000.0,40,39,2000.0,0,5,0.00,0.0,0.0


In [14]:
# Visualizando a correlação em tabela
# Coeficiente de correlação: 
# +1  = forte correlação positiva
# 0   = não há correlação
# -1  = forte correlação negativa
df_dados.corr()

,SpcPrimeiraConsultaQtdConsulta,SpcPrimeiraConsultaSpcQtdRestricao,AtrasoPrimeiroPagamento,VendaVlrVenda,VendaQtdVenda,VendaVlrPago,VendaVlrAtraso,VendaVlrAVencer,PagamentoVlrPago,PagamentoQtdPago,...,PrimeiroLimiteCredito_Ind,LimiteCredito_Ind,IdadeAtual_Ind,IdadePrimeiraCompra_Ind,Renda_Ind,PrazoMediaAtraso,PrazoMedioVenda,VlrDh,VlrCartaoCredito,VlrCartaoDebito
SpcPrimeiraConsultaQtdConsulta,1.000000,-0.010049,0.065821,0.023494,-0.000074,-0.003922,0.062429,0.038279,-0.003922,-0.017333,...,0.056044,-0.012994,0.028238,0.032976,0.029160,-0.007911,0.016263,-0.015493,0.018606,0.022682
SpcPrimeiraConsultaSpcQtdRestricao,-0.010049,1.000000,0.022157,0.001011,0.031024,-0.005557,0.008526,0.015629,-0.005557,0.017771,...,-0.133556,-0.054903,0.023605,0.024508,-0.003675,0.008695,0.002234,0.020401,0.009225,0.010596
AtrasoPrimeiroPagamento,0.065821,0.022157,1.000000,0.120495,0.084203,-0.039657,0.354711,0.231238,-0.039657,-0.017099,...,-0.074905,-0.275235,-0.105928,-0.104041,-0.000125,0.101093,0.202080,0.018913,-0.017731,0.007173
VendaVlrVenda,0.023494,0.001011,0.120495,1.000000,0.800085,0.933982,0.062874,0.429282,0.933982,0.803520,...,0.142983,0.329646,0.095155,0.087617,0.017146,-0.084180,-0.041944,0.203420,0.092398,0.138898
VendaQtdVenda,-0.000074,0.031024,0.084203,0.800085,1.000000,0.795794,-0.048144,0.271759,0.795794,0.894291,...,-0.009696,0.244171,-0.002019,-0.009520,0.006815,-0.040126,-0.227515,0.191459,0.098133,0.114351
VendaVlrPago,-0.003922,-0.005557,-0.039657,0.933982,0.795794,1.000000,-0.213449,0.227848,1.000000,0.854704,...,0.120793,0.394895,0.104433,0.094868,0.016411,-0.028299,-0.158064,0.197196,0.100112,0.139170
VendaVlrAtraso,0.062429,0.008526,0.354711,0.062874,-0.048144,-0.213449,1.000000,-0.095578,-0.213449,-0.188910,...,-0.019034,-0.448338,-0.058578,-0.060665,0.002207,-0.135605,0.256225,0.005226,-0.023682,-0.003874
VendaVlrAVencer,0.038279,0.015629,0.231238,0.429282,0.271759,0.227848,-0.095578,1.000000,0.227848,0.224009,...,0.143920,0.358756,0.056861,0.065316,0.004485,-0.077130,0.136766,0.068342,0.020990,0.037936
PagamentoVlrPago,-0.003922,-0.005557,-0.039657,0.933982,0.795794,1.000000,-0.213449,0.227848,1.000000,0.854704,...,0.120793,0.394895,0.104433,0.094868,0.016411,-0.028299,-0.158064,0.197196,0.100112,0.139170
PagamentoQtdPago,-0.017333,0.017771,-0.017099,0.803520,0.894291,0.854704,-0.188910,0.224009,0.854704,1.000000,...,0.033904,0.333114,0.047447,0.038721,0.005946,-0.048123,-0.072876,0.171659,0.087115,0.107973


In [15]:
# Visualizando correlação com variavel target
corr_matrix = df_dados.corr().abs()
print(corr_matrix['AtrasoPrimeiroPagamento'].sort_values(ascending=False).head(100))

AtrasoPrimeiroPagamento               1.000000
AbertoVlr                             0.441474
AbertoQtd                             0.426678
AbertoVlrAtraso                       0.354711
VendaVlrAtraso                        0.354711
AbertoQtdAtraso                       0.340763
AbertoVlrAtrasoAcima                  0.281045
LimiteCredito_Ind                     0.275235
AbertoQtdAtrasoAcima                  0.273552
AbertoVlrAVencer                      0.231238
VendaVlrAVencer                       0.231238
AbertoQtdAVencer                      0.224335
PrazoMedioVenda                       0.202080
AbertoVlrPrevPerda                    0.193573
AbertoQtdAtraso30                     0.189462
AbertoVlrAtraso30                     0.188522
AbertoQtdPrevPerda                    0.182748
AbertoVlrCano                         0.173647
AbertoQtdCano                         0.170901
PagamentoVlrPontual                   0.169115
AbertoQtdAtraso60                     0.151940
AbertoVlrAtra

In [16]:
#Alta correlação ou colineridade pode causar overfitting
# Índice de features com alta correlação
#nesse caso rever o fit e ajustar conforme resultados dos treinos e aumento do volume de dados após análise da amostra
to_drop = [column for column in corr_matrix.columns if any(corr_matrix[column] > 0.50)]
print('Columns to drop: ' , (len(to_drop)))

Columns to drop:  51


In [17]:
#Pré-processamento 
#Redução de dimensionalidade
# PCA - mantém 90% da variancia
pca = PCA(0.90)
principal_components = pca.fit_transform(df_dados)
principal_df = pd.DataFrame(data = principal_components)
print(principal_df.shape)
#somente uma feature explica a variância dos dados, entender melhor e ver se c volume muda o comportamento

(45349, 1)


In [18]:
#Split variáveis preditoras e target
df_dados_target = df_dados['AtrasoPrimeiroPagamento']
df_dados = df_dados.drop(['AtrasoPrimeiroPagamento'], axis=1)

In [19]:
#Pré-processamento 
#Normalização
#Colocando os dados na mesma escala
scaler = StandardScaler()

In [20]:
#virgulas foram substituídas por ponto no arquivo fonte antes da importação para agilizar(fazer por aqui)
nparray_dados = scaler.fit_transform(df_dados)
nparray_dados

array([[-0.65986759, -0.76820108, -1.28248539, ..., -0.08409446,
        -0.10108451, -0.1355839 ],
       [ 0.82366498, -0.76820108, -1.28248539, ..., -0.22287621,
        -0.10108451, -0.1355839 ],
       [-0.65986759,  1.30174251, -0.61073555, ..., -0.18962641,
        -0.10108451, -0.1355839 ],
       ...,
       [ 0.82366498, -0.76820108, -0.61073555, ..., -0.22287621,
        -0.10108451, -0.1355839 ],
       [ 0.82366498,  1.30174251, -0.61073555, ..., -0.22287621,
        -0.10108451, -0.1355839 ],
       [ 0.82366498, -0.76820108,  0.73276415, ..., -0.22287621,
        -0.10108451, -0.1355839 ]])

In [21]:
#Criação dos modelos
# Definindo a taxa de split
split_test_size = 0.30
# Criando dados de treino e de teste
x_treino, x_teste, y_treino, y_teste = train_test_split(df_dados, df_dados_target, test_size = split_test_size, random_state = 42)

In [22]:
modelo = RandomForestClassifier(random_state = 42)
modelo.fit(x_treino, y_treino.ravel())

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [23]:
# Verificando os dados de treino
rf_predict_train = modelo.predict(x_treino)
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(y_treino, rf_predict_train)))

Exatidão (Accuracy): 1.0000


In [24]:
# Verificando nos dados de teste
rf_predict_test = modelo.predict(x_teste)
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(y_teste, rf_predict_test)))
print()

Exatidão (Accuracy): 0.9213



In [25]:
print("Confusion Matrix")

print("{0}".format(metrics.confusion_matrix(y_teste, rf_predict_test, labels = [1, 0])))
print("")

#F1 Score: quanto maior melhor

print("Classification Report")
print(metrics.classification_report(y_teste, rf_predict_test, labels = [1, 0]))

Confusion Matrix
[[9898  603]
 [ 468 2636]]

Classification Report
              precision    recall  f1-score   support

           1       0.95      0.94      0.95     10501
           0       0.81      0.85      0.83      3104

    accuracy                           0.92     13605
   macro avg       0.88      0.90      0.89     13605
weighted avg       0.92      0.92      0.92     13605



In [26]:
# Versão do modelo usando Regressão Logística
modelo_v2 = LogisticRegression(C = 0.7, random_state = 42)
modelo_v2.fit(x_treino, y_treino.ravel())
lr_predict_test = modelo_v2.predict(x_teste)

C:\Users\diogo\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [27]:
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(y_teste, lr_predict_test)))
print()
print("Classification Report")
print(metrics.classification_report(y_teste, lr_predict_test, labels = [1, 0]))

Exatidão (Accuracy): 0.8961

Classification Report
              precision    recall  f1-score   support

           1       0.93      0.94      0.93     10501
           0       0.78      0.76      0.77      3104

    accuracy                           0.90     13605
   macro avg       0.85      0.85      0.85     13605
weighted avg       0.90      0.90      0.90     13605



In [28]:
lr_predict_test

array([1, 1, 0, ..., 0, 1, 1], dtype=int64)

In [ ]:
#gerando arquivo do modelo com melhor performance para produção
#from sklearn.externals import joblib
#joblib.dump(rf_predict_train, 'random_forest.pk1')

In [31]:
# Grid Search Random Forest

# Usando um grid completo de todos os parâmetros
param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# Executando o Grid Search
grid_search = GridSearchCV(modelo, param_grid = param_grid, return_train_score = True)

grid_search.fit(x_treino, y_treino)

grid_search.cv_results_

{'mean_fit_time': array([0.73317485, 0.71081614, 0.74617119, 1.07042661, 0.91298337,
        1.38033094, 1.5757184 , 1.55342054, 1.61191673, 2.81837149,
        1.43299494, 1.20003014, 2.29521818, 2.30401974, 1.80876431,
        3.49505496, 3.33069725, 3.68184228, 0.64308043, 0.6313189 ,
        0.64049416, 0.90079226, 0.97857699, 0.92454886, 1.76867299,
        1.8085712 , 1.61667805, 1.8081717 , 1.50916615, 1.31586804,
        2.52664723, 2.01881866, 1.87598481, 5.22563   , 4.68747621,
        3.85270123, 0.75897632, 0.97649078, 0.68477726, 0.99355011,
        1.03065171, 1.02047186, 1.82693696, 1.93442798, 1.92864351,
        1.99666882, 1.50418577, 1.29335299, 2.82604456, 2.43829527,
        2.095398  , 5.54158363, 5.6060204 , 4.64937   , 0.97439504,
        0.75059319, 0.75638537, 1.3888885 , 1.12339635, 1.24088244,
        2.53501558, 2.90643024, 2.66846929, 2.98586001, 2.32689095,
        1.50976439, 3.4056962 , 2.90005488, 2.47718177, 6.30396228,
        6.13346024, 5.32894731]

In [32]:
grid_search.best_params_

{'bootstrap': False,
 'criterion': 'entropy',
 'max_depth': None,
 'max_features': 10,
 'min_samples_leaf': 10}

In [ ]:
#Executando Random Forest com parâmetros recomendados pelo GridSearch
modelo = RandomForestClassifier(random_state = 42)
modelo.fit(x_treino, y_treino.ravel())
# Verificando os dados de treino
rf_predict_train = modelo.predict(x_treino)
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(y_treino, rf_predict_train)))

print("Classification Report")
print(metrics.classification_report(y_teste, rf_predict_test, labels = [1, 0]))

In [ ]:
#MODELO PARA TREINAR VÁRIOS MODELOS E VER QUAL É O MELHOR MODELO
#sklearn.ensemble.GradientBoostingClassifier